In [77]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")

In [78]:
# with help from http://planspace.org/20150423-forward_selection_with_statsmodels/
import statsmodels.formula.api as smf

def forward_selected(data, response):
    """Linear model designed by forward selection.

    Parameters:
    -----------
    data : pandas DataFrame with all possible predictors and response

    response: string, name of response column in data

    Returns:
    --------
    model: an "optimal" fitted statsmodels linear model
           with an intercept
           selected by forward selection
           evaluated by adjusted R-squared
    """
    
    remaining = set(data.columns)
    remaining.remove(response)
    selected = []
    current_score, best_new_score = 0.0, 0.0
    while remaining and current_score == best_new_score:
        scores_with_candidates = []
        for candidate in remaining:
            formula = "{} ~ {} + 1".format(response, ' + '.join(selected + [candidate]))
            score = smf.ols(formula, data).fit().rsquared_adj
            scores_with_candidates.append((score, candidate))
        scores_with_candidates.sort()
        best_new_score, best_candidate = scores_with_candidates.pop()
        if current_score < (best_new_score * 1):
            remaining.remove(best_candidate)
            selected.append(best_candidate)
            current_score = best_new_score
    formula = "{} ~ {} + 1".format(response, ' + '.join(selected))
    
    model = smf.ols(formula, data).fit()
    return model

In [79]:
response=pd.read_pickle('data/ZipcodeMVPSnew.pkl')[['RegionName','2000_agg','2010_agg']]
response=response.rename(columns={'RegionName': 'zipcode'})
response['2010_agg_adj'] = response['2010_agg']/1.26
response['pct_delta'] = (response['2010_agg_adj']/response['2000_agg']) - 1
response = response.drop(['2010_agg','2000_agg', '2010_agg_adj'], 1)

## Testing a few different response variables:
### - house price delta (above)
### - house/rental ratio delta (below)
### - house/rental ratio 2015 (below)

In [80]:
ratio_response = pd.read_pickle('data/housing_rental_ratio.pkl')
ratio_response = ratio_response.rename(columns = {'RegionName': 'zipcode'})

ratio_delta_response = ratio_response[['zipcode','ratio_pct_delta']]
ratio_2015_response = ratio_response[['zipcode','ratio_2015']]

ratio_delta_response['zipcode'] = ratio_delta_response['zipcode'].astype('str')
ratio_2015_response['zipcode'] = ratio_2015_response['zipcode'].astype('str')

/Users/princess_eliza/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/princess_eliza/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [81]:
ratio_delta_response.head(3)

,zipcode,ratio_pct_delta
0,11368,0.984804
1,79936,1.001354
2,60629,1.073514


In [82]:
ratio_2015_response.head(3)

,zipcode,ratio_2015
0,11368,197.083754
1,79936,108.096860
2,60629,96.111037


In [83]:
response.head(3)

,zipcode,pct_delta
1,79936,0.173843
2,60629,0.085907
3,90650,0.426587


In [84]:
# training and test sets that were normalized, and training and test sets that were normalized and transformed ('_t')
train_set = pd.read_pickle('data/train_all_features_norm.pkl')
itrain_df = pd.DataFrame(train_set.index).rename(columns={0: 'ind'})

train_set_t = pd.read_pickle('data/train_all_features_norm_and_transform.pkl')
itrain_t_df = pd.DataFrame(train_set_t.index).rename(columns={0: 'ind'})

test_set = pd.read_pickle('data/test_all_features_norm.pkl')
itest_df = pd.DataFrame(test_set.index).rename(columns={0: 'ind'})

test_set_t = pd.read_pickle('data/test_all_features_norm_and_transform.pkl')
itest_t_df = pd.DataFrame(test_set_t.index).rename(columns={0: 'ind'})

features = pd.concat([train_set, test_set])
features_t = pd.concat([train_set_t, test_set_t])

print features.shape, train_set.shape, test_set.shape
features.head(10)

(27653, 64) (22122, 64) (5531, 64)


,zipcode,prison,jail,A001,A002,A003,A004,A005,A006,A007,A008,A009,A010,A011,A012,A013,A014,A015,A016,A017,A018,A019,A020,A021,A022,A023,A024,A025,A026,A027,A028,A029,A030,A031,A032,A033,A034,A035,A036,A037,A038,A039,A040,A041,A042,A043,A044,A045,A046,A047,A048,A049,A050,A051,A052,A053,A054,A055,A056,n_establishments,paid_employees,first_quarter_payroll_1000,annual_payroll_1000,median_income
0,01001,0,0,0.393159,-0.844788,-0.576017,-0.174118,-0.117701,0.107539,-0.016376,0.170177,0.302420,1.862902,-0.661973,-0.790923,-0.635659,-0.198994,-0.050199,-0.043662,-0.090009,-0.044475,-0.020593,1.102067,0.725830,-0.772259,-0.455802,-0.057597,-0.171337,0.315046,0.117725,0.383984,0.672488,2.097870,0.537700,-0.396109,-0.176487,-0.012918,-0.132979,-0.367899,-0.890846,-0.317130,-0.457076,-0.174062,-0.299229,1.122282,0.268374,0.452518,1.600822,1.382658,-0.747222,0.588576,0.758788,-0.797671,-0.140852,-0.530444,-0.652569,-0.228583,0.486598,-1.127662,0.459736,0.742016,0.244980,0.334447,0.334018
1,01002,0,0,1.245532,-1.599994,1.060064,3.597249,-1.353027,-2.046088,-1.219645,-0.868623,-0.806043,-0.218702,-0.400512,-1.502592,0.492105,2.820575,-1.110719,-1.885773,-1.384025,-0.915197,-0.790664,-0.376806,0.464421,-1.508369,1.426019,3.888342,-1.261798,-1.604638,-0.859521,-0.715226,-0.731682,-0.143021,-0.223040,-0.195494,-0.162262,1.524147,-0.132979,-0.065958,-1.612656,-1.106050,-0.983398,-0.122170,0.166579,0.116185,-0.327456,-0.507041,0.520319,0.090974,-0.843631,-0.630502,0.751994,-0.790133,-0.264675,-0.530444,-0.166848,-0.330651,0.973388,-0.031431,0.763299,1.633192,0.335312,0.417431,0.110293
3,01005,0,0,-0.418261,-0.321953,0.815872,-0.337381,-0.276076,0.681839,0.556609,-0.311218,-0.667485,0.078670,0.165987,-0.298230,1.099363,-0.272642,-0.387638,0.416865,0.479358,0.001353,-0.469801,-0.202821,-0.101965,-0.262645,0.416261,-0.352621,-0.107192,0.778418,0.552056,-0.565334,-0.865412,0.193112,0.667700,-0.433724,-0.176487,-0.316536,-0.132979,-0.466359,0.196325,0.156222,0.244687,-0.139468,-0.066325,-0.223221,-0.142543,0.185974,-0.233994,0.090974,0.291852,-0.171839,0.602543,-0.624302,-0.471047,0.240861,0.226354,0.230722,-0.461362,-0.442518,-0.416119,-0.399458,-0.271116,-0.314831,0.984848
4,01007,0,0,0.246007,0.171836,0.254233,-0.435338,0.008999,1.327927,0.757154,-0.361891,-0.736764,-0.553246,-0.247993,0.194464,0.231852,-0.346290,-0.194816,0.704695,0.582879,-0.365267,-0.790664,-0.463799,0.311932,0.133722,0.232669,-0.463255,0.341821,1.572770,0.823513,-0.365477,-0.664817,-0.535177,0.547330,-0.421186,-0.190712,-0.012918,-0.003289,-0.413847,0.321082,-0.685293,-0.457076,-0.104873,0.195692,-0.502019,-0.656190,-0.507041,-0.172834,-0.339587,0.538230,-0.835693,0.779167,-0.820284,-0.677419,-0.221922,0.249484,0.154171,0.191188,-0.481669,-0.062791,-0.286955,-0.233152,-0.263373,1.043897
6,01009,0,0,-0.719368,-0.496231,-0.087635,0.201386,0.832550,0.574158,-0.102324,-0.108525,-1.256356,0.487556,0.492813,-0.133999,0.492105,0.169246,0.962115,-0.043662,-0.245291,0.230490,-0.983182,0.145149,-0.428727,-0.828883,-0.685293,0.200548,0.341821,1.043202,0.009142,-0.365477,-1.333469,0.585268,0.658071,-0.477609,-0.190712,-0.335512,-0.132979,-0.453231,-0.561130,-0.001562,-0.457076,0.759997,0.894404,0.528321,0.658746,-0.027262,0.234903,0.029466,-0.286602,-0.618432,0.548198,-0.564000,-0.388498,-0.838966,-0.375014,-0.177549,-0.754192,-0.227187,-0.617665,-0.509473,-0.307238,-0.358285,-0.233389
9,01012,0,0,-0.724921,-1.193344,-0.160892,-0.435338,-0.592827,0.466476,1.817176,0.727583,-0.563567,0.190184,0.144198,-1.283617,0.058350,-0.125346,-0.725076,-0.331492,2.083938,0.642937,-0.469801,0.667104,-0.080181,-0.998755,-0.409904,-0.795156,-0.171337,1.307986,1.257844,0.733733,-0.531086,-0.086999,0.720663,-0.490147,-0.176487,-0.259608,-0.132979,-0.486050,-0.222503,0.839952,2.525414,-0.398929,-0.677697,0.128307,0.206737,-0.773585,-0.009739,-0.831657,-0.393723,-0.654642,-0.124331,0.182238,-0.759968,-0.838966,-0.398144,-0.407202,-0.753849,0.203475,-0.622642,-0.515830,-0.312899,-0.364498,-0.092024
11,01020,0,0

In [85]:
merged = response.merge(features, how='inner', on=['zipcode'])
merged_t = response.merge(features_t, how='inner', on=['zipcode'])

merged = merged.astype('float')
merged_t = merged_t.astype('float')

merged = merged.rename(columns={'pct_delta' : 'response'})
merged_t = merged_t.rename(columns={'pct_delta' : 'response'})

merged_t.shape, merged.shape

((11602, 65), (11602, 65))

In [86]:
merged_ratio_delta = ratio_delta_response.merge(features_t, how='inner', on=['zipcode'])
merged_ratio_2015 = ratio_2015_response.merge(features_t, how='inner', on=['zipcode'])

merged_ratio_delta = merged_ratio_delta.astype('float')
merged_ratio_2015 = merged_ratio_2015.astype('float')

merged_ratio_delta = merged_ratio_delta.rename(columns={'ratio_pct_delta':'response'})
merged_ratio_2015 = merged_ratio_2015.rename(columns={'ratio_2015':'response'})

merged_ratio_delta.shape, merged_ratio_2015.shape, features_t.shape

((10306, 65), (10306, 65), (27653, 64))

In [87]:
# house price delta response (_t ones are better - only use them)
merged_train = merged.merge(itrain_df, how='inner', right_on='ind', left_index=True)
merged_test = merged.merge(itest_df, how='inner', right_on='ind', left_index=True)
merged_t_train = merged_t.merge(itrain_t_df, how='inner', right_on='ind', left_index=True)
merged_t_test = merged_t.merge(itest_t_df, how='inner', right_on='ind', left_index=True)

# house/rental ratio delta response and house/rental ratio 2015 response
ratio_delta_train = merged_ratio_delta.merge(itrain_t_df, how='inner', right_on='ind', left_index=True)
ratio_delta_test = merged_ratio_delta.merge(itest_t_df, how='inner', right_on='ind', left_index=True)

ratio_2015_train = merged_ratio_2015.merge(itrain_t_df, how='inner', right_on='ind', left_index=True)
ratio_2015_test = merged_ratio_2015.merge(itest_t_df, how='inner', right_on='ind', left_index=True)

In [88]:
merged_t_train.shape, merged_t_train.shape, ratio_2015_test.shape, ratio_2015_train.shape, ratio_delta_test.shape, ratio_delta_train.shape

((9271, 66), (9271, 66), (2098, 66), (8208, 66), (2098, 66), (8208, 66))

In [89]:
import time
# house delta response
start_time = time.time()
bestmodel = forward_selected(merged_train.drop('zipcode',1), 'response')

print '%s seconds to train model'%(time.time() - start_time)

90.3473331928 seconds to train model


In [110]:
bestmodel.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               response   R-squared:                       0.390
Model:                            OLS   Adj. R-squared:                  0.387
Method:                 Least Squares   F-statistic:                     120.3
Date:                Tue, 08 Dec 2015   Prob (F-statistic):               0.00
Time:                        21:14:49   Log-Likelihood:                 2154.7
No. Observations:                9271   AIC:                            -4209.
Df Residuals:                    9221   BIC:                            -3853.
Df Model:                          49                                         
Covariance Type:            nonrobust                                         
==============================================================================================
                                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
----------------------------------------------------------------------------------------------
Intercept                     -0.5131      0.567     -0.905      0.366        -1.625     0.599
A002                          -0.0550      0.008     -6.773      0.000        -0.071    -0.039
A054                          -0.0208      0.011     -1.906      0.057        -0.042     0.001
A003                          -0.0848      0.016     -5.374      0.000        -0.116    -0.054
median_income                  0.0532      0.004     13.315      0.000         0.045     0.061
A050                          16.3152     12.800      1.275      0.202        -8.776    41.407
A052                          -0.0617      0.004    -16.821      0.000        -0.069    -0.055
A024                          -0.1257      0.077     -1.637      0.102        -0.276     0.025
A051                          -0.0435      0.003    -12.660      0.000        -0.050    -0.037
zipcode                    -1.765e-06   9.04e-08    -19.528      0.000     -1.94e-06 -1.59e-06
A036                           0.0224      0.005      4.621      0.000         0.013     0.032
A035                           0.0120      0.002      5.559      0.000         0.008     0.016
A031                          -0.1226      0.015     -8.274      0.000        -0.152    -0.094
ind                         6.755e-06   1.73e-06      3.896      0.000      3.36e-06  1.02e-05
A032                          -0.0991      0.012     -8.111      0.000        -0.123    -0.075
A044                           0.0389      0.010      4.085      0.000         0.020     0.058
A029                           0.0985      0.044      2.237      0.025         0.012     0.185
A040                           0.0805      0.016      5.194      0.000         0.050     0.111
annual_payroll_1000           -0.0106      0.016     -0.674      0.500        -0.041     0.020
n_establishments               0.0554      0.005     10.665      0.000         0.045     0.066
paid_employees                -0.0505      0.007     -7.022      0.000        -0.065    -0.036
A041                          -0.0564      0.013     -4.470      0.000        -0.081    -0.032
A019                           0.1859      0.046      4.061      0.000         0.096     0.276
A037                          -0.2038      0.016    -12.690      0.000        -0.235    -0.172
A042                          -0.5743      0.327     -1.756      0.079        -1.215     0.067
A048                           0.1330      0.016      8.513      0.000         0.102     0.164
A047                           0.0934      0.015      6.030      0.000         0.063     0.124
A046                          -0.0629      0.011     -5.791      0.000        -0.084    -0.042
A034                          -0.0184      0.005     -4.044      0.000        -0.027    -0.009
A009                          -0.3239      

In [107]:
import time
start_time = time.time()
# house/rental ratio delta response
bestmodel_ratio_delta = forward_selected(ratio_delta_train.drop('zipcode',1), 'response')
# house/rental ratio response
bestmodel_ratio_2015 = forward_selected(ratio_2015_train.drop('zipcode',1), 'response')

print '%s seconds to train 2 models'%(time.time() - start_time)

120.388909817 seconds to train 2 models


In [108]:
bestmodel_ratio_delta.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               response   R-squared:                       0.285
Model:                            OLS   Adj. R-squared:                  0.281
Method:                 Least Squares   F-statistic:                     80.11
Date:                Tue, 08 Dec 2015   Prob (F-statistic):               0.00
Time:                        21:14:49   Log-Likelihood:                 4941.6
No. Observations:                7686   AIC:                            -9805.
Df Residuals:                    7647   BIC:                            -9534.
Df Model:                          38                                         
Covariance Type:            nonrobust                                         
==============================================================================================
                                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
----------------------------------------------------------------------------------------------
Intercept                      1.1131      0.012     94.749      0.000         1.090     1.136
A036                           0.0092      0.003      3.616      0.000         0.004     0.014
A056                           0.0472      0.004     10.536      0.000         0.038     0.056
A052                           0.0279      0.002     12.791      0.000         0.024     0.032
A034                           0.0125      0.003      4.916      0.000         0.008     0.017
A039                           0.0569      0.006      9.368      0.000         0.045     0.069
A002                          -0.0567      0.006    -10.170      0.000        -0.068    -0.046
A032                          -0.0229      0.003     -7.270      0.000        -0.029    -0.017
ind                        -1.126e-05   1.58e-06     -7.132      0.000     -1.44e-05 -8.16e-06
A044                           0.0060      0.006      0.970      0.332        -0.006     0.018
A054                           0.0892      0.015      6.147      0.000         0.061     0.118
A038                          -0.0158      0.007     -2.340      0.019        -0.029    -0.003
A025                           0.0056      0.004      1.572      0.116        -0.001     0.013
A051                           0.0178      0.003      6.014      0.000         0.012     0.024
A014                          -0.0275      0.007     -4.077      0.000        -0.041    -0.014
A037                          -0.0743      0.011     -6.495      0.000        -0.097    -0.052
n_establishments              -0.0335      0.009     -3.902      0.000        -0.050    -0.017
A049                           0.0422      0.006      6.943      0.000         0.030     0.054
A040                          -0.0451      0.010     -4.713      0.000        -0.064    -0.026
A042                          -0.0532      0.009     -5.758      0.000        -0.071    -0.035
A031                          -0.0195      0.004     -4.591      0.000        -0.028    -0.011
A050                           0.0221      0.006      3.688      0.000         0.010     0.034
A035                          -0.0042      0.001     -3.322      0.001        -0.007    -0.002
A041                           0.0206      0.008      2.504      0.012         0.004     0.037
median_income                  0.0040      0.003      1.357      0.175        -0.002     0.010
A010                           0.0271      0.006      4.221      0.000         0.015     0.040
A030                          -0.0220      0.006     -3.738      0.000        -0.034    -0.010
A053                          -0.0637      0.018     -3.477      0.001        -0.100    -0.028
A047                           0.0262      0.012      2.179      0.029         0.003     0.050
A018                           0.0228      

In [109]:
bestmodel_ratio_2015.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               response   R-squared:                       0.639
Model:                            OLS   Adj. R-squared:                  0.637
Method:                 Least Squares   F-statistic:                     351.9
Date:                Tue, 08 Dec 2015   Prob (F-statistic):               0.00
Time:                        21:14:49   Log-Likelihood:                -38445.
No. Observations:                8208   AIC:                         7.697e+04
Df Residuals:                    8166   BIC:                         7.727e+04
Df Model:                          41                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------------
Intercept          115.6556      3.048     37.944      0.000       109.681   121.631
A034                 5.4867      0.493     11.124      0.000         4.520     6.454
A041                -8.0052      0.812     -9.862      0.000        -9.596    -6.414
median_income       10.8612      0.600     18.112      0.000         9.686    12.037
A036                 3.7798      0.502      7.527      0.000         2.795     4.764
A018                30.6848      9.065      3.385      0.001        12.915    48.455
A032               -18.3619      0.628    -29.257      0.000       -19.592   -17.132
A031               -18.2574      0.791    -23.071      0.000       -19.809   -16.706
A037               -40.0220      2.277    -17.579      0.000       -44.485   -35.559
A042               -26.8470      1.847    -14.534      0.000       -30.468   -23.226
A025                 5.7860      0.701      8.254      0.000         4.412     7.160
A050                -9.2643      1.202     -7.710      0.000       -11.620    -6.909
A001                 5.0654      0.676      7.488      0.000         3.739     6.391
A056                10.2582      0.897     11.440      0.000         8.500    12.016
A038                -8.4858      1.204     -7.049      0.000       -10.846    -6.126
A052                -2.6617      0.445     -5.988      0.000        -3.533    -1.790
A006                 3.9180      0.870      4.502      0.000         2.212     5.624
A030                 0.6901      1.074      0.642      0.521        -1.416     2.796
A009                -4.5629      3.015     -1.514      0.130       -10.472     1.346
A048                14.4308      1.972      7.317      0.000        10.565    18.297
A044                -6.1830      1.350     -4.581      0.000        -8.829    -3.537
ind                  0.0015      0.000      3.944      0.000         0.001     0.002
n_establishments    22.3858      1.855     12.069      0.000        18.750    26.022
paid_employees     -16.8251      1.553    -10.837      0.000       -19.869   -13.782
A051                 2.5801      0.587      4.393      0.000         1.429     3.731
A049                -5.9319      1.240     -4.783      0.000        -8.363    -3.501
A055                -8.4790      1.892     -4.482      0.000       -12.188    -4.770
A035                 0.8657      0.258      3.350      0.001         0.359     1.372
jail                 3.4001      1.118      3.042      0.002         1.209     5.591
A039                 2.1103      1.128      1.871      0.061        -0.100     4.321
A028                18.0308      8.342      2.161      0.031         1.678    34.383
A015                 4.0682      0.834      4.881      0.000         2.434     5.702
A027                 4.6354      1.012      4.581      0.000         2.652     6.619
A024                 3.

In [113]:
import time
start_time = time.time()
bestmodel_t = forward_selected(merged_t_train.drop('zipcode',1), 'response')

print '%s seconds to train model'%(time.time() - start_time)

69.0808401108 seconds to train model


In [114]:
bestmodel_t.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               response   R-squared:                       0.372
Model:                            OLS   Adj. R-squared:                  0.369
Method:                 Least Squares   F-statistic:                     121.5
Date:                Tue, 08 Dec 2015   Prob (F-statistic):               0.00
Time:                        21:21:22   Log-Likelihood:                 2020.2
No. Observations:                9271   AIC:                            -3948.
Df Residuals:                    9225   BIC:                            -3620.
Df Model:                          45                                         
Covariance Type:            nonrobust                                         
==============================================================================================
                                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
----------------------------------------------------------------------------------------------
Intercept                      0.1053      0.019      5.431      0.000         0.067     0.143
A002                          -0.0688      0.009     -7.814      0.000        -0.086    -0.052
A036                           0.0493      0.004     13.665      0.000         0.042     0.056
A052                          -0.0674      0.004    -17.275      0.000        -0.075    -0.060
median_income                  0.0475      0.004     11.838      0.000         0.040     0.055
A049                          -0.0785      0.008     -9.323      0.000        -0.095    -0.062
A035                           0.0163      0.002      9.038      0.000         0.013     0.020
A025                           0.0136      0.006      2.403      0.016         0.003     0.025
A055                          -0.0600      0.014     -4.416      0.000        -0.087    -0.033
A040                           0.0869      0.013      6.457      0.000         0.060     0.113
A051                          -0.0460      0.004    -11.482      0.000        -0.054    -0.038
A037                          -0.1698      0.016    -10.677      0.000        -0.201    -0.139
A053                           0.0981      0.027      3.571      0.000         0.044     0.152
A003                          -0.0761      0.008     -9.218      0.000        -0.092    -0.060
A042                          -0.5755      0.331     -1.737      0.082        -1.225     0.074
A044                           0.0495      0.010      5.081      0.000         0.030     0.069
A056                          -0.0456      0.006     -7.340      0.000        -0.058    -0.033
n_establishments               0.1399      0.013     10.791      0.000         0.115     0.165
paid_employees                -0.1074      0.011     -9.724      0.000        -0.129    -0.086
A024                           0.0278      0.009      2.967      0.003         0.009     0.046
ind                         9.475e-06   2.07e-06      4.587      0.000      5.43e-06  1.35e-05
A034                           0.0193      0.004      5.405      0.000         0.012     0.026
A006                           0.0043      0.012      0.348      0.728        -0.020     0.028
A048                           0.0957      0.016      6.013      0.000         0.064     0.127
A029                           0.1063      0.045      2.386      0.017         0.019     0.194
A047                           0.0800      0.017      4.772      0.000         0.047     0.113
A010                          -0.0506      0.010     -5.075      0.000        -0.070    -0.031
A030                           0.0334      0.008      4.095      0.000         0.017     0.049
A033                           0.0333      0.010      3.216      0.001         0.013     0.054
A046                          -0.0365      

## get features from forward selection to use in sklearn regressions

In [143]:
# house price delta response
best_features = bestmodel.params.index.values.tolist() + ['response', 'zipcode']
best_features_t = bestmodel_t.params.index.values.tolist() + ['response', 'zipcode']

# house/rental ratio delta response
best_features_ratio_delta = bestmodel_ratio_delta.params.index.values.tolist() + ['response', 'zipcode']
# house/rental ratio 2015 response
best_features_ratio_2015 = bestmodel_ratio_2015.params.index.values.tolist() + ['response', 'zipcode']

In [144]:
best_features

['Intercept',
 'A002',
 'A054',
 'A003',
 'median_income',
 'A050',
 'A052',
 'A024',
 'A051',
 'zipcode',
 'A036',
 'A035',
 'A031',
 'ind',
 'A032',
 'A044',
 'A029',
 'A040',
 'annual_payroll_1000',
 'n_establishments',
 'paid_employees',
 'A041',
 'A019',
 'A037',
 'A042',
 'A048',
 'A047',
 'A046',
 'A034',
 'A009',
 'A045',
 'A020',
 'A055',
 'A011',
 'A028',
 'A056',
 'A016',
 'A033',
 'A014',
 'A004',
 'first_quarter_payroll_1000',
 'A043',
 'jail',
 'A038',
 'A049',
 'A013',
 'A008',
 'A025',
 'A039',
 'prison',
 'response',
 'zipcode']

In [145]:
type(best_features)

list

In [146]:
len(merged_train.columns), len(best_features), len(best_features_t)

(66, 52, 48)

In [158]:
# house response
bad_features_t = [el for el in merged_t_train.columns if (el not in best_features_t)]
merged_train_t_best = merged_t_train.drop(bad_features_t + ['ind'], 1)
merged_test_t_best = merged_t_test.drop(bad_features_t + ['ind'], 1)

In [159]:
len(best_features_t), len(merged_t_train.columns)

(48, 66)

In [163]:
# house/rental ratio response
bad_features_ratio_delta = [el for el in ratio_delta_train.columns if (el not in best_features_ratio_delta)]
bad_features_ratio_2015 = [el for el in ratio_2015_train.columns if (el not in best_features_ratio_2015)]

ratio_delta_train_best = ratio_delta_train.drop(bad_features_ratio_delta + ['ind'], 1).dropna()
ratio_delta_test_best = ratio_delta_test.drop(bad_features_ratio_delta + ['ind'], 1).dropna()

ratio_2015_train_best = ratio_2015_train.drop(bad_features_ratio_2015 + ['ind'], 1).dropna()
ratio_2015_test_best = ratio_2015_test.drop(bad_features_ratio_2015 + ['ind'], 1).dropna()

In [164]:
ratio_delta_train_best.shape, ratio_delta_test_best.shape, ratio_2015_train_best.shape, ratio_2015_test_best.shape

((7686, 39), (1970, 39), (8208, 42), (2098, 42))

In [165]:
ratio_delta_train_best.shape, ratio_delta_test_best.shape, ratio_2015_train_best.shape, ratio_2015_test_best.shape

((7686, 39), (1970, 39), (8208, 42), (2098, 42))

In [166]:
merged_train_t_best.head(2)

,zipcode,response,prison,jail,A002,A003,A006,A008,A009,A010,A017,A019,A024,A025,A026,A028,A029,A030,A031,A033,A034,A035,A036,A037,A040,A041,A042,A043,A044,A045,A046,A047,A048,A049,A050,A051,A052,A053,A054,A055,A056,n_establishments,paid_employees,first_quarter_payroll_1000,annual_payroll_1000,median_income
0,79936,0.173843,0,0,1.043228,1.353093,0.251114,-1.274009,-1.290995,-0.924961,-0.918179,-1.239872,0.495572,1.047414,0.646026,-1.164903,-1.199738,-1.299125,0.008071,-0.119587,-0.020352,-0.003289,2.706301,1.256761,1.590272,1.826019,-1.362656,-1.087653,-1.200055,-1.212563,-1.354481,2.230742,-1.402987,0.860686,-1.673460,-0.729692,-0.376183,1.637257,1.481053,1.805892,1.064799,1.825619,1.803127,1.142390,1.422776,-0.054825
1,60629,0.085907,0,0,1.711295,1.035645,-0.574443,-1.426029,-1.325635,-0.813446,-1.332265,-1.239872,0.864351,1.175703,-0.479306,-1.314796,-1.266604,-0.929836,-1.994890,-0.105362,-0.396648,-0.132979,2.470766,0.909223,2.040004,1.796906,-0.938398,-0.758919,-0.507041,-0.825213,-0.862411,2.359287,-0.871903,0.290055,-0.022089,1.099626,0.240861,2.909382,3.165172,1.779076,1.573763,1.350804,1.178124,0.356887,0.430216,-0.457911


## Linear Regression with Lasso Regularization (normalized and transformed features)

In [167]:
from sklearn.linear_model import LassoCV
# HOUSE RESPONSE
start_time = time.time()

X_train = merged_train_t_best.drop(['zipcode','response'], 1).values
Y_train = merged_train_t_best['response'].values
X_test = merged_test_t_best.drop(['zipcode','response'], 1).values
Y_test = merged_test_t_best['response'].values

alphas = [0.001, 0.01, 0.1, 1.0, 10.0, 100.0]
alphas_temp = [0.001, 0.01]
clf_lasso_model = LassoCV(alphas = alphas).fit(X_train, Y_train)
best_alpha_lasso = clf_lasso_model.alpha_

print '----- %s seconds to run ----- \n---- best alpha = %s ----'%(time.time() - start_time, best_alpha_lasso)

----- 0.184986829758 seconds to run ----- 
---- best alpha = 0.001 ----


In [168]:
# calculate accuracy
start_time = time.time()

training_accuracy = clf_lasso_model.score(X_train, Y_train)
test_accuracy = clf_lasso_model.score(X_test, Y_test)
print '#### based on the best Linear Regression with Lasso Regularization ####'
print "R-squared on training data: %0.2f" % (training_accuracy)
print "R-squared on test data:     %0.2f" % (test_accuracy)
print '----- %s seconds to run -----'%(time.time() - start_time)

#### based on the best Linear Regression with Lasso Regularization ####
R-squared on training data: 0.36
R-squared on test data:     0.33
----- 0.00258708000183 seconds to run -----


In [169]:
# HOUSE/RENTAL RATIO DELTA RESPONSE
start_time = time.time()

X_train = ratio_delta_train_best.drop(['response','zipcode'], 1).values
Y_train = ratio_delta_train_best['response'].values
X_test = ratio_delta_test_best.drop(['response','zipcode'], 1).values
Y_test = ratio_delta_test_best['response'].values

alphas = [0.001, 0.01, 0.1, 1.0, 10.0, 100.0]
alphas_temp = [0.001, 0.01]
lasso_model_ratio_delta = LassoCV(alphas = alphas).fit(X_train, Y_train)
best_alpha_lasso = lasso_model_ratio_delta.alpha_

print '----- %s seconds to run ----- \n---- best alpha = %s ----'%(time.time() - start_time, best_alpha_lasso)

----- 0.0450799465179 seconds to run ----- 
---- best alpha = 0.001 ----


In [170]:
# calculate accuracy
start_time = time.time()

training_accuracy = lasso_model_ratio_delta.score(X_train, Y_train)
test_accuracy = lasso_model_ratio_delta.score(X_test, Y_test)
print '#### based on the best Linear Regression with Lasso Regularization ####'
print "R-squared on training data: %0.2f" % (training_accuracy)
print "R-squared on test data:     %0.2f" % (test_accuracy)
print '----- %s seconds to run -----'%(time.time() - start_time)

#### based on the best Linear Regression with Lasso Regularization ####
R-squared on training data: 0.26
R-squared on test data:     0.26
----- 0.00198602676392 seconds to run -----


In [171]:
# HOUSE/RENTAL RATIO 2015 RESPONSE
start_time = time.time()

X_train = ratio_2015_train_best.drop(['response','zipcode'], 1).values
Y_train = ratio_2015_train_best['response'].values
X_test = ratio_2015_test_best.drop(['response','zipcode'], 1).values
Y_test = ratio_2015_test_best['response'].values

alphas = [0.001, 0.01, 0.1, 1.0, 10.0, 100.0]
alphas_temp = [0.001, 0.01]
lasso_model_ratio_2015 = LassoCV(alphas = alphas).fit(X_train, Y_train)
best_alpha_lasso = lasso_model_ratio_2015.alpha_

print '----- %s seconds to run ----- \n---- best alpha = %s ----'%(time.time() - start_time, best_alpha_lasso)

----- 0.391252040863 seconds to run ----- 
---- best alpha = 0.1 ----


In [172]:
# calculate accuracy
start_time = time.time()

training_accuracy = lasso_model_ratio_2015.score(X_train, Y_train)
test_accuracy = lasso_model_ratio_2015.score(X_test, Y_test)
print '#### based on the best Linear Regression with Lasso Regularization ####'
print "R-squared on training data: %0.2f" % (training_accuracy)
print "R-squared on test data:     %0.2f" % (test_accuracy)
print '----- %s seconds to run -----'%(time.time() - start_time)

#### based on the best Linear Regression with Lasso Regularization ####
R-squared on training data: 0.63
R-squared on test data:     0.63
----- 0.00206398963928 seconds to run -----


## Linear Regression with Ridge Regularization (normalized and transformed features)

In [175]:
from sklearn.linear_model import RidgeCV

start_time = time.time()
X_train = merged_train_t_best.drop(['zipcode','response'], 1).values
Y_train = merged_train_t_best['response'].values
X_test = merged_test_t_best.drop(['zipcode','response'], 1).values
Y_test = merged_test_t_best['response'].values

alphas = [0.001, 0.01, 0.1, 1.0, 10.0, 100.0]
clf_ridge_model = RidgeCV(alphas = alphas).fit(X_train, Y_train)
best_alpha_ridge = clf_ridge_model.alpha_

print '----- %s seconds to run ----- \n---- best alpha = %s ----'%(time.time() - start_time, best_alpha_ridge)

----- 0.0724050998688 seconds to run ----- 
---- best alpha = 0.1 ----


In [176]:
training_accuracy = clf_ridge_model.score(X_train, Y_train)
test_accuracy = clf_ridge_model.score(X_test, Y_test)
print '#### based on the best Linear Regression with Ridge Regularization ####'
print "R-squared on training data: %0.2f" % (training_accuracy)
print "R-squared on test data:     %0.2f" % (test_accuracy)

#### based on the best Linear Regression with Ridge Regularization ####
R-squared on training data: 0.37
R-squared on test data:     0.33


In [177]:
# HOUSE/RENTAL RATIO DELTA RESPONSE
start_time = time.time()

X_train = ratio_delta_train_best.drop(['response','zipcode'], 1).values
Y_train = ratio_delta_train_best['response'].values
X_test = ratio_delta_test_best.drop(['response','zipcode'], 1).values
Y_test = ratio_delta_test_best['response'].values

alphas = [0.001, 0.01, 0.1, 1.0, 10.0, 100.0]
ridge_model_ratio_delta = RidgeCV(alphas = alphas).fit(X_train, Y_train)
best_alpha_ridge = ridge_model_ratio_delta.alpha_

print '----- %s seconds to run ----- \n---- best alpha = %s ----'%(time.time() - start_time, best_alpha_ridge)

----- 0.0404560565948 seconds to run ----- 
---- best alpha = 1.0 ----


In [178]:
training_accuracy = ridge_model_ratio_delta.score(X_train, Y_train)
test_accuracy = ridge_model_ratio_delta.score(X_test, Y_test)
print '#### based on the best Linear Regression with Ridge Regularization ####'
print "R-squared on training data: %0.2f" % (training_accuracy)
print "R-squared on test data:     %0.2f" % (test_accuracy)

#### based on the best Linear Regression with Ridge Regularization ####
R-squared on training data: 0.28
R-squared on test data:     0.26


In [179]:
# HOUSE/RENTAL RATIO 2015 RESPONSE
start_time = time.time()

X_train = ratio_2015_train_best.drop(['response','zipcode'], 1).values
Y_train = ratio_2015_train_best['response'].values
X_test = ratio_2015_test_best.drop(['response','zipcode'], 1).values
Y_test = ratio_2015_test_best['response'].values

alphas = [0.001, 0.01, 0.1, 1.0, 10.0, 100.0]
ridge_model_ratio_2015 = RidgeCV(alphas = alphas).fit(X_train, Y_train)
best_alpha_ridge = ridge_model_ratio_2015.alpha_

print '----- %s seconds to run ----- \n---- best alpha = %s ----'%(time.time() - start_time, best_alpha_ridge)

----- 0.0534300804138 seconds to run ----- 
---- best alpha = 1.0 ----


In [180]:
training_accuracy = ridge_model_ratio_2015.score(X_train, Y_train)
test_accuracy = ridge_model_ratio_2015.score(X_test, Y_test)
print '#### based on the best Linear Regression with Ridge Regularization ####'
print "R-squared on training data: %0.2f" % (training_accuracy)
print "R-squared on test data:     %0.2f" % (test_accuracy)

#### based on the best Linear Regression with Ridge Regularization ####
R-squared on training data: 0.64
R-squared on test data:     0.63


In [ ]:
from sklearn.svm import LinearSVC

clfsvm=LinearSVC(loss="hinge")
Cs=[0.001, 0.01, 0.1, 1.0, 10.0, 100.0]
Xmatrix=dftouse[lcols].values
Yresp=dftouse['RESP'].values

Xmatrix_train=Xmatrix[mask]
Xmatrix_test=Xmatrix[~mask]
Yresp_train=Yresp[mask]
Yresp_test=Yresp[~mask]

#your code here
from sklearn.grid_search import GridSearchCV
import time

start_time = time.time()

n_folds = 5
n_jobs = 4
gs = GridSearchCV(clfsvm, param_grid={"C": Cs}, cv=n_folds, n_jobs=n_jobs)
gs.fit(Xmatrix_train, Yresp_train)

C_best = gs.best_params_['C']
print 'The best c-value is %s'%(C_best)

print '----- %s seconds to run -----'%(time.time() - start_time)

In [ ]:
##### In case we want to do SVM regression/other types of models later

In [ ]:
#calculate the accuracy here
#your code here
start_time = time.time()

best = LinearSVC(C=C_best, class_weight=None, dual=True, fit_intercept=True,
      intercept_scaling=1, loss='hinge', max_iter=1000, multi_class='ovr',
      penalty='l2', random_state=None, tol=0.0001, verbose=0)
best.fit(Xmatrix_train, Yresp_train)

training_accuracy = best.score(Xmatrix_train, Yresp_train)
test_accuracy = best.score(Xmatrix_test, Yresp_test)
print '#### based on the best LinearSVC ####'
print "Accuracy on training data: %0.2f" % (training_accuracy)
print "Accuracy on test data:     %0.2f" % (test_accuracy)
print '----- %s seconds to run -----'%(time.time() - start_time)